In [1]:
import sys, os

# Enable importing pysparky
sys.path.append(os.pardir)

In [2]:
import pyspark
from pyspark.sql import SparkSession, DataFrame, Column
from pyspark.sql import functions as F, types as T

print(pyspark.__version__)

spark = SparkSession.builder.getOrCreate()

3.5.5


25/06/16 14:56:13 WARN Utils: Your hostname, codespaces-66b7ee resolves to a loopback address: 127.0.0.1; using 10.0.1.198 instead (on interface eth0)
25/06/16 14:56:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/16 14:56:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Start Spark session
spark = SparkSession.builder.appName("SampleSplit").getOrCreate()

# Define schema
schema = StructType([
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("age", IntegerType(), True)
])

# Sample data
data = [
    ("Alice", "Engineering", 29),
    ("Bob", "Marketing", 35),
    ("Charlie", "Engineering", 32),
    ("Diana", "HR", 28),
    ("Ethan", "Marketing", 41),
    ("Fiona", "HR", 30)
]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()


25/06/16 15:01:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------+-----------+---+
|   name| department|age|
+-------+-----------+---+
|  Alice|Engineering| 29|
|    Bob|  Marketing| 35|
|Charlie|Engineering| 32|
|  Diana|         HR| 28|
|  Ethan|  Marketing| 41|
|  Fiona|         HR| 30|
+-------+-----------+---+



In [18]:
from pyspark.sql import DataFrame
from typing import Union

def split_dataframe_by_column_groups(
    df: DataFrame,
    column_name: str,
    *value_groups: list[Union[str, int]]
) -> tuple[DataFrame, ...]:
    """
    Splits a DataFrame into multiple DataFrames based on groups of values in a specified column.

    Parameters:
    df (DataFrame): The input PySpark DataFrame.
    column_name (str): The column to split on.
    *value_groups (list): Variable number of value groups (each a list of values).

    Returns:
    tuple: A tuple of DataFrames, one for each group of values.
    """
    return tuple(
        df.filter(df[column_name].isin(group))
        for group in value_groups
    )


In [20]:
# Sample usage with the earlier DataFrame
group1 = ["HR", "Engineering"]
group2 = "Marketing"

df1, df2 = split_dataframe_by_column_groups(df, "department", group1, group2)

df1.show()  # Shows rows where department is HR or Engineering
df2.show()  # Shows rows where department is Marketing


+-------+-----------+---+
|   name| department|age|
+-------+-----------+---+
|  Alice|Engineering| 29|
|Charlie|Engineering| 32|
|  Diana|         HR| 28|
|  Fiona|         HR| 30|
+-------+-----------+---+

+-----+----------+---+
| name|department|age|
+-----+----------+---+
|  Bob| Marketing| 35|
|Ethan| Marketing| 41|
+-----+----------+---+

